In [2]:
import numpy as np
import pandas as pd
from scipy.sparse import csr_matrix
from scipy.sparse import spdiags
from scipy.stats import multivariate_normal
from copy import deepcopy
from sklearn.metrics import pairwise_distances
from sklearn.preprocessing import normalize

In [3]:
wiki = pd.read_csv('people_wiki.csv')

In [4]:
wiki.head()

,URI,name,text
0,<http://dbpedia.org/resource/Digby_Morrell>,Digby Morrell,digby morrell born 10 october 1979 is a former...
1,<http://dbpedia.org/resource/Alfred_J._Lewy>,Alfred J. Lewy,alfred j lewy aka sandy lewy graduated from un...
2,<http://dbpedia.org/resource/Harpdog_Brown>,Harpdog Brown,harpdog brown is a singer and harmonica player...
3,<http://dbpedia.org/resource/Franz_Rottensteiner>,Franz Rottensteiner,franz rottensteiner born in waidmannsfeld lowe...
4,<http://dbpedia.org/resource/G-Enka>,G-Enka,henry krvits born 30 december 1974 in tallinn ...


In [5]:
def load_sparse_csr(filename):
    loader = np.load(filename)
    data = loader['data']
    indices = loader['indices']
    indptr = loader['indptr']
    shape = loader['shape']
    
    return csr_matrix( (data, indices, indptr), shape)

In [6]:
tf_idf = load_sparse_csr('4_tf_idf.npz')

In [7]:
%ls

3_em-for-gmm_blank.ipynb
4_map_index_to_word.gl/
4_map_index_to_word.json
4_tf_idf.npz
Implement EM for Gaussian mixtures.ipynb
Untitled.ipynb
_167fc84b78dc145609e919983b475aaa_people_wiki.csv.zip
_8022d9156b93197ae8878c79eba9767b_4_map_index_to_word.gl.zip
_8022d9156b93197ae8878c79eba9767b_4_map_index_to_word.json.zip
_a1fd39587bbfe4e686cae5cfb37f2c1c_chosen_images.png
_af95de312b911daa1d7142fd60ae12e1_4_tf_idf.npz.zip
images/
people_wiki.csv


In [8]:
map_index_to_word = pd.read_json('4_map_index_to_word.json', typ='series')

In [9]:
wiki.loc[0]

URI           <http://dbpedia.org/resource/Digby_Morrell>
name                                        Digby Morrell
text    digby morrell born 10 october 1979 is a former...
Name: 0, dtype: object

In [10]:
tf_idf = normalize(tf_idf)

## EM in high dimensions

### Log probability function for diagonal covariance Gaussian.

In [11]:
def diag(array):
    n = len(array)
    return spdiags(array, 0, n, n)

def logpdf_diagonal_gaussian(x, mean, cov):
    '''
    Compute logpdf of a multivariate Gaussian distribution with diagonal covariance at a given point x.
    A multivariate Gaussian distribution with a diagonal covariance is equivalent
    to a collection of independent Gaussian random variables.

    x should be a sparse matrix. The logpdf will be computed for each row of x.
    mean and cov should be given as 1D numpy arrays
    mean[i] : mean of i-th variable
    cov[i] : variance of i-th variable'''

    n = x.shape[0]
    dim = x.shape[1]
    assert(dim == len(mean) and dim == len(cov))

    # multiply each i-th column of x by (1/(2*sigma_i)), where sigma_i is sqrt of variance of i-th variable.
    scaled_x = x.dot( diag(1./(2*np.sqrt(cov))) )
    # multiply each i-th entry of mean by (1/(2*sigma_i))
    scaled_mean = mean/(2*np.sqrt(cov))

    # sum of pairwise squared Eulidean distances gives SUM[(x_i - mean_i)^2/(2*sigma_i^2)]
    return -np.sum(np.log(np.sqrt(2*np.pi*cov))) - pairwise_distances(scaled_x, [scaled_mean], 'euclidean').flatten()**2

In [12]:
def log_sum_exp(x, axis):
    '''Compute the log of a sum of exponentials'''
    x_max = np.max(x, axis=axis)
    if axis == 1:
        return x_max + np.log( np.sum(np.exp(x-x_max[:,np.newaxis]), axis=1) )
    else:
        return x_max + np.log( np.sum(np.exp(x-x_max), axis=0) )

def EM_for_high_dimension(data, means, covs, weights, cov_smoothing=1e-5, maxiter=int(1e3), thresh=1e-4, verbose=False):
    # cov_smoothing: specifies the default variance assigned to absent features in a cluster.
    #                If we were to assign zero variances to absent features, we would be overconfient,
    #                as we hastily conclude that those featurese would NEVER appear in the cluster.
    #                We'd like to leave a little bit of possibility for absent features to show up later.
    n = data.shape[0]
    dim = data.shape[1]
    mu = deepcopy(means)
    Sigma = deepcopy(covs)
    K = len(mu)
    weights = np.array(weights)

    ll = None
    ll_trace = []

    for i in range(maxiter):
        # E-step: compute responsibilities
        logresp = np.zeros((n,K))
        for k in range(K):
            logresp[:,k] = np.log(weights[k]) + logpdf_diagonal_gaussian(data, mu[k], Sigma[k])
        ll_new = np.sum(log_sum_exp(logresp, axis=1))
        if verbose:
            print(ll_new)
        logresp -= np.vstack(log_sum_exp(logresp, axis=1))
        resp = np.exp(logresp)
        counts = np.sum(resp, axis=0)

        # M-step: update weights, means, covariances
        weights = counts / np.sum(counts)
        for k in range(K):
            mu[k] = (diag(resp[:,k]).dot(data)).sum(axis=0)/counts[k]
            mu[k] = mu[k].A1

            Sigma[k] = diag(resp[:,k]).dot( data.multiply(data)-2*data.dot(diag(mu[k])) ).sum(axis=0) \
                       + (mu[k]**2)*counts[k]
            Sigma[k] = Sigma[k].A1 / counts[k] + cov_smoothing*np.ones(dim)

        # check for convergence in log-likelihood
        ll_trace.append(ll_new)
        if ll is not None and (ll_new-ll) < thresh and ll_new > -np.inf:
            ll = ll_new
            break
        else:
            ll = ll_new

    out = {'weights':weights,'means':mu,'covs':Sigma,'loglik':ll_trace,'resp':resp}

    return out

## Initialize mean params using k-means

In [37]:
from sklearn.cluster import KMeans

np.random.seed(5)
num_clusters = 25

# Use scikit-learn's k-means to simplify workflow
kmeans_model = KMeans(n_clusters=num_clusters, n_init=5, max_iter=400, random_state=1, n_jobs=-1)
kmeans_model.fit(tf_idf)
centroids, cluster_assignment = kmeans_model.cluster_centers_, kmeans_model.labels_

means = [centroid for centroid in centroids]

## Initializing cluster weights

In [38]:
cluster_assignment

array([22, 23, 15, ...,  2,  2,  8], dtype=int32)

In [39]:
num_docs = tf_idf.shape[0]
weights = []
for i in range(num_clusters):
    # Compute the number of data points assigned to cluster i:
    num_assigned = (cluster_assignment == i).sum() # YOUR CODE HERE
    w = float(num_assigned) / num_docs
    weights.append(w)
    # **Initializing covariances.
    # ** To initialize our covariance parameters, 
    # we compute:
    # $\hat{\sigma}_{k, j}^2 = \sum_{i=1}^{N}(x_{i,j} - \hat{\mu}_{k, j})^2$ 
    # for each feature $j$. 
    # For features with really tiny variances, we assign 1e-8 instead to prevent numerical instability.
    # We do this computation in a vectorized fashion in the following code block.

## Initialize covariances

In [40]:
covs = []
for i in range(num_clusters):
    member_rows = tf_idf[cluster_assignment==i]
    cov = (member_rows.multiply(member_rows) - 2*member_rows.dot(diag(means[i]))).sum(axis=0).A1 / member_rows.shape[0] \
          + means[i]**2
    cov[cov < 1e-8] = 1e-8
    covs.append(cov)

In [41]:
len(covs)

25

## Running EM

In [42]:
out = EM_for_high_dimension(tf_idf, means, covs, weights, cov_smoothing=1e-10)
print (out['loglik']) # print history of log-likelihood over time

[3855847476.7012835, 4844053202.46348, 4844053202.46348]


## Intepret clusters

In [43]:
map_index_to_word

sundayswadsworth       37318
verplank               46431
damfunk                23253
anyer                  62437
sonja                  37922
degussa                33295
heraldleader           55678
vinalop                38962
chameleons             15539
cfe                    19068
spiders                76384
loanstopshop            6262
hanging                85150
woody                  94338
suzana                 67596
comically              57840
localized              44264
yougoslavia             7189
sprague                16230
gteborg                82999
chanthaburi            17970
igual                  52391
bandmeyns              10335
unnecessarily          65916
storiesin              84637
canek                  27791
petrak                 44943
naturopathic           42153
bratislava             86932
heroesin               23118
                       ...  
511spouse              57845
percent                97449
ep                     97672
branco        

In [44]:
map_index_to_word.iloc[0]

37318

In [45]:
map_index_to_word

sundayswadsworth       37318
verplank               46431
damfunk                23253
anyer                  62437
sonja                  37922
degussa                33295
heraldleader           55678
vinalop                38962
chameleons             15539
cfe                    19068
spiders                76384
loanstopshop            6262
hanging                85150
woody                  94338
suzana                 67596
comically              57840
localized              44264
yougoslavia             7189
sprague                16230
gteborg                82999
chanthaburi            17970
igual                  52391
bandmeyns              10335
unnecessarily          65916
storiesin              84637
canek                  27791
petrak                 44943
naturopathic           42153
bratislava             86932
heroesin               23118
                       ...  
511spouse              57845
percent                97449
ep                     97672
branco        

In [46]:
map_index_to_word.keys()

Index(['sundayswadsworth', 'verplank', 'damfunk', 'anyer', 'sonja', 'degussa',
       'heraldleader', 'vinalop', 'chameleons', 'cfe',
       ...
       '1941his', 'tieon', 'expands', '35mm', 'vous', 'gal', 'goole', 'agios',
       'trocadero', 'corey'],
      dtype='object', length=100282)

In [47]:
for u in map_index_to_word.keys():
    print (u, map_index_to_word[u])

sundayswadsworth 37318
verplank 46431
damfunk 23253
anyer 62437
sonja 37922
degussa 33295
heraldleader 55678
vinalop 38962
chameleons 15539
cfe 19068
spiders 76384
loanstopshop 6262
hanging 85150
woody 94338
suzana 67596
comically 57840
localized 44264
yougoslavia 7189
sprague 16230
gteborg 82999
chanthaburi 17970
igual 52391
bandmeyns 10335
unnecessarily 65916
storiesin 84637
canek 27791
petrak 44943
naturopathic 42153
bratislava 86932
heroesin 23118
theoryhe 74282
broward 85834
screaming 85122
petras 70455
receiving 99209
grueling 74796
vibrational 53741
kangema 42859
dcduring 33588
elgan 57644
wednesday 93707
millermost 6231
nielsenfunk 24347
crotch 13410
elgar 70186
598m 20672
196769 68867
shows 99850
mcelhinnycolleen 2399
bienniale 54484
vintntso 10023
jhey 18000
abdulwahab 44419
270 86553
271 83371
272 68057
273 67642
274 10850
selassie 83506
gorman 48569
277 77728
gwenno 48174
reengineering 23974
finalsst 1990
targa 21188
prosody 30971
musicafter 81084
mugariro 38530
balliol 880

oldies 87795
climbs 28492
honour 97745
universityshu 8949
vanderbilt 93273
bredbennerturning 44679
bowmans 58593
306 67617
earthhe 4945
address 97390
dwindling 44382
artistshyboys 7060
awardheenan 15673
benson 93652
flodin 1496
rosterlater 15149
lengle 56357
impacted 85460
cette 13585
terminator2004 30442
queue 78841
panarab 39839
matilda 68013
fichandlers 30169
giraud 25098
raise 97883
ellensburg 46324
houseman 80602
paced 71699
mnchengladbach 21377
womenswear 70567
zinnias 48150
albumas 1013
h1674 58868
opposes 85816
handbags 14906
rayos 58628
vigour 26918
selfpublished 89221
institutes2 28194
israelstern 2308
pavilions 51170
chelle 64802
nondecisions 27831
magazinesoutside 26329
kozlov 11987
rehovot 47428
concertgebouw 90638
wvu 31551
violationduring 5614
consoles 10271
goods 94134
jidaigeki 81767
vonneumann 3317
bharati 79793
198995 22041
198992 10835
198993 3757
198990 90264
198991 49316
symposiumskennedy 63044
rebounding 90295
pointsterlich 6848
neosoul 37614
gregoire 69142
ruthe

angelproduced 47605
heenan 22090
harpsichord 90129
sealed 83150
brazilian 97607
sese 20210
dantone 42718
bubble 89890
witt 70419
emergence 93562
wits 67510
bohemians 84649
fomamolf 951
societal 85562
cowritersmulherons 26384
societas 64687
statesman 92510
internalist 47282
parasites 68343
with 100269
abused 93591
pardubice 69569
naming 94161
monopoly 81633
kompa 4706
marginality 11756
rags 79664
caylor 19097
postcoup 60283
operationally 52000
dirty 95038
abuser 68751
abuses 87043
hemmerts 58963
cirpsuniversit 37391
stockin 39799
spraysipper 18407
tripp 70880
munda 13048
deathucker 1603
ohiofox 49487
methylmercury 33936
mundi 78406
biologistshe 39498
merkelo 33320
marchapril 61298
watches 60297
psgeg 59722
creac 31685
almosameh 35150
ensuing 91086
formulation 84101
mundy 57146
watched 94359
oneals 67709
theile 12301
stationduring 31327
cream 87018
pamina 21863
yoga 92559
pitchedbauer 37261
strueby 35089
grouppreviously 55668
noticiero 70069
yogo 40761
yogi 67748
bhagam 61050
sympathetic

3627 37851
cipher 51409
garmintransitions 38449
neria 66609
manon 81660
guillot 40461
ulukalala 49318
manoj 4550
instil 66321
brek 26353
tebuan 38326
bred 90449
tremayne 56455
brea 18570
lots 86643
battignano 10845
taskforce 73217
undersea 68454
bret 68352
alassads 1631
foys 60878
anlaug 53747
raghavendra 82857
promising 96869
scotts 83316
disputing 39863
kenyas 88199
scotty 73844
venkata 21821
mechanicalaeronautical 6340
kabarett 7179
cottage 80831
scotto 738
scotti 8125
admiralwatson 42734
sculptorceramicist 39227
birthtown 7655
reynards 31777
mmin 28624
entities 87875
tape 95968
riding 98317
cappelen 33634
biondi 29102
carouge 35971
menlucas 32548
abbf 18899
gryphonwood 47443
abba 77025
changeux 8888
lithographically 20023
iro 57388
computerimplemented 30962
chronologies 44890
changeup 80941
singlefamily 10625
guistina 29185
caedo 55677
swingman 84402
ira 92239
californiawith 23018
belzer 59104
illinoisafter 51717
dominican 95824
aes 71900
aer 45009
ekiden 64901
circulation 89182
gl

cvitani 20510
samsons 69214
backrow 73010
applegates 23547
danseur 10361
niels 84468
dewesternization 15911
golub 31455
stormthe 39178
canrio 33048
ethiopias 68696
amsterdamhis 37380
historianlott 6037
haslam 24198
regionnaires 55580
ossetia 78327
rncm 12167
emilyspencer 46848
schola 40056
scholl 77482
eleganceeduardo 32805
saeeda 54681
dolce 74346
13underpar 56326
azrd 27536
gyanendras 67356
sadiq 13997
selvi 37072
duncans 6904
nantwich 33660
selva 38602
ferraris 14950
natos 75433
millionjason 47099
bandmasters 71234
jacquet 73749
opie 73748
kuppepadavu 935
guesthost 37575
barfly 41509
marakwet 53746
processthe 5473
synagogues 81122
trotta 38192
pauley 75118
rooie 57887
rendering 79368
superstock 34987
antalya 76226
500yard 2840
ecospirituality 55551
vflafl 80234
wanamaker 70665
tysons 22869
crowe 84027
sovereign 90274
221b 65640
spinnin 80260
boom 89543
persboekhandel 15298
mosques 82728
infomercial 67697
freelanced 70678
nettwerk 71886
presentationin 41053
relationssattler 57931
2su

...

petov 25712
serpentine 7913
interweaves 42706
waitingfollowing 16807
xfm 82304
eiji 83760
ktvx 17079
ktvu 56258
rabun 49376
tasksshe 33680
stagereturning 54811
fundamentalists 86392
cancin 62103
29198 4028
jumpsuit 68415
levitt 67141
hushdrops 11779
lkin 25709
palike 42499
roecker 38451
elfman 43753
12260 42637
grim 81665
wayland 74855
aacsb 40047
crimeladen 3328
snap 39164
sinisa 54810
snat 41282
herriman 19017
silken 59800
bio 82820
bil 43373
qwl 59057
akureyri 27405
ligia 49994
bii 58406
big 99721
bid 97620
bic 57125
synodbishop 36273
redeem 81245
biz 2717
50377 7917
magnum 91294
macarthur 89908
khabarovsk 41320
catalogsat 39868
bir 78939
bis 84664
dlit 39035
secondshe 11225
bahaa 11558
at40 21988
alltel 47112
lachance 17286
associate 99588
warrenpenned 53568
princis 3572
bahat 40108
musicalromantic 22920
bulleen 71644
yearswhile 51205
ladenburgs 4708
lexingtonfayette 52231
google 95471
often 99813
extremism 86126
magnificent 90984
extremist 81926
accelerate 78871
interestsmore 1518

heel 84676
saunton 13960
tawana 55651
leuven 87462
healy 80110
cockinesshe 26967
transcendentalists 35061
swimsuits 56691
langhorne 21644
abolitionist 15662
bigname 67190
ironpigs 84306
geraint 65512
ecclesiastic 65480
ratlam 45043
festwochen 3443
highlighting 91755
turkmenistan 31512
faltriqueira 39205
corpse 84179
chainz 50861
lifesized 48129
fiddle 91603
cyprushe 59564
chains 85064
simulacrumhis 49852
regain 91509
hose 49575
accident 97216
athletissima 43169
globalin 76898
diwekar 59026
2011his 5140
florito 11492
millonarios 18501
1968discovery 13123
christened 44695
beaker 70871
ranieri 48005
33062 7400
bhamra 11591
albanian 87535
visconti 68898
azania 23995
trax 11774
camagey 51489
blessings 61402
penaltyon 32252
rightsas 37812
conway 91185
chronic 93808
lobbies 80476
103rd 81299
mayabazaar 34811
chronik 50796
officein 74573
3215 943
utopias 42227
buckle 61620
redbeck 36665
awardee 47493
famebrotherman 11660
applegate 73053
utopian 68195
officesin 60656
maze 85844
oise 12601
inter

34th 86918
hydrographic 30916
tricksters 58281
1991that 38704
10tvhe 24532
actifi 24402
pfeiffer 37417
gartels 34235
vocabularies 53451
wakaw 40234
keenan 85868
inamdar 59653
basuki 34157
mahmoud 81548
singerhe 83095
wilks 1531
combustion 78975
temptation 87197
karlssons 52516
gravityhe 51976
paternal 90068
stageartioli 14399
attemptsbrady 63716
rtl 83130
awardment 3315
designin 26492
newswriters 61308
cajun 79132
bargains 8115
171951 38566
mooneyhan 61089
switchhitting 9147
charpentiers 31246
20075 67762
dedication 94771
satirizing 31328
sizemores 47280
hills 97494
adapting 85469
playera 78302
annanias 58145
2ser 83413
jelly 84193
kammersngerin 69606
facet 77421
players 99709
batallas 35230
facer 9931
offhe 19030
diminishes 37050
sretenovion 34952
frequenting 57675
cabinetin 27157
sullivan 94493
confide 29696
betting 85568
unicorns 74358
titletrack 36570
karel 90469
karen 95646
reuniting 78805
agustn 84475
woodvillewest 69491
comical 15812
bathroom 71627
1978rapson 37520
boylum 44921


markevitch 3483
fables 30441
teldec 42593
worldhe 86265
inspect 53523
actressher 4173
krogstad 60191
voracious 38485
cupoutside 8692
icomosperu 16285
koekelare 18083
bbk 19135
2008packwood 31011
affording 61363
torino 86178
baffled 62745
stahlhelm 47733
comedyking 4605
peoplelyttle 20318
vedder 21404
heines 51036
coloradojames 51880
hansons 17725
confluence 54144
spainhis 70992
racehorses 46742
looney 73501
hypothermia 17841
jian 59393
soupe 50706
levitin 4842
macleodin 42464
revolutionaries 3072
allireland 92427
marjorie 84229
fantastique 48528
austriagreen 6861
artus 60152
artur 80833
harrell 76559
chaptersdr 19805
susanto 17659
1999during 13150
imprisonmenthe 48073
cnncoms 4818
ferrie 17655
calamities 4000
distinctions 90755
teamsin 48354
reindeer 72244
brentano 75091
ferris 81918
eitheroverall 46476
ferriz 24682
expatriate 81001
valley 98955
heartsmart 64518
jtc1 2609
hard 98875
tinniswood 66484
fundamentals 89876
farnborough 58952
solovyovs 60445
cabinet 98891
staiswaf 68927
2950 

gorewhen 2743
oshkosh 40808
rajshahi 70624
kalamata 73263
undersheriff 45058
helaine 60467
srvn 35505
highwater 65011
niupat 20089
newsas 44628
wm 84573
dagsavisen 75126
hometownon 8552
vietnameseamerican 41210
metro 96015
warehousing 75191
brownsvilleprice 1257
architektur 54622
metre 91543
evading 36829
bickert 49390
breeds 44376
paraclete 37953
seasonin 93463
cognitionockenfels 52515
initials 88727
parisi 33596
apple 96255
grouptogether 66183
19962000 86194
dedicated 99016
romerosamson 4598
herzog 86458
priester 66363
hippodrome 58690
motor 96095
himwalker 62530
socrates 81143
vancura 45814
recordscurrently 14247
bourjeilys 25278
bachelets 56854
ciashe 12453
selfhatred 1684
ices 50430
vella 70080
gyurcsny 39706
votesalbanese 7384
weeping 74339
expanding 94883
rjfcroach 15150
automobile 94673
porco 43326
nock 72243
friendsyardley 39572
osadebe 34838
refurbished 15763
carroll 95459
listswoope 48788
cooperate 71082
8 99897
penis 12915
zoboomafoo 44043
secondgrade 26517
sugarhis 56438
i

In [13]:
map_index_to_word_2 = {map_index_to_word[key]: key for key in map_index_to_word.keys()}

In [49]:
map_index_to_word[0]

37318

In [50]:
type(map_index_to_word)

pandas.core.series.Series

In [14]:
# Fill in the blanks
def visualize_EM_clusters(tf_idf, means, covs, map_index_to_word_2):
    print('')
    print('==========================================================')

    num_clusters = len(means)
    for c in range(num_clusters):
        print('Cluster {0:d}: Largest mean parameters in cluster '.format(c))
        print('\n{0: <12}{1: <12}{2: <12}'.format('Word', 'Mean', 'Variance'))
        
        # The k'th element of sorted_word_ids should be the index of the word 
        # that has the k'th-largest value in the cluster mean. Hint: Use np.argsort().
        sorted_word_ids = np.argsort(means[c])[::-1]  # YOUR CODE HERE

        for i in sorted_word_ids[:5]:
            print ('{0: <12}{1:<10.2e}{2:10.2e}'.format(map_index_to_word_2[i], 
                                                       means[c][i],
                                                       covs[c][i]))
        print ('\n=====================================================' + 
               '\nQuiz Question. Select all the topics that have a cluster in the model created above. [multiple choice]====')


In [52]:
'''By EM'''
visualize_EM_clusters(tf_idf, out['means'], out['covs'], map_index_to_word_2)


Cluster 0: Largest mean parameters in cluster 

Word        Mean        Variance    
minister    7.57e-02    7.42e-03
election    5.89e-02    3.21e-03
party       5.89e-02    2.61e-03
liberal     2.93e-02    4.55e-03
elected     2.91e-02    8.95e-04

Quiz Question. Select all the topics that have a cluster in the model created above. [multiple choice]====
Cluster 1: Largest mean parameters in cluster 

Word        Mean        Variance    
film        1.76e-01    6.07e-03
films       5.50e-02    2.97e-03
festival    4.66e-02    3.60e-03
feature     3.69e-02    1.81e-03
directed    3.39e-02    2.22e-03

Quiz Question. Select all the topics that have a cluster in the model created above. [multiple choice]====
Cluster 2: Largest mean parameters in cluster 

Word        Mean        Variance    
art         1.26e-01    6.83e-03
museum      5.62e-02    7.27e-03
gallery     3.65e-02    3.40e-03
artist      3.61e-02    1.44e-03
design      3.20e-02    4.59e-03

Quiz Question. Select all the to

## Comparing to random initialization


In [ ]:
np.random.seed(5)
#num_clusters = len(means)
num_clusters = 25
num_docs, num_words = tf_idf.shape

random_means = []
random_covs = []
random_weights = []

for k in range(num_clusters):
    
    # Create a numpy array of length num_words with random normally distributed values.
    # Use the standard univariate normal distribution (mean 0, variance 1).
    mean = np.random.randn(num_words)
    
    # Create a numpy array of length num_words with random values uniformly distributed between 1 and 5.
    cov = np.random.rand(num_words, num_words) * 4 + 1

    # Initially give each cluster equal weight.
    weight = [1. / num_clusters] * num_clusters
    
    random_means.append(mean)
    random_covs.append(cov)
    random_weights.append(weight)

## Quiz Question: Try fitting EM with the random initial parameters you created above. (Use cov_smoothing=1e-5.) Store the result to out_random_init. What is the final loglikelihood that the algorithm converges to?



In [ ]:
out_random_init = EM_for_high_dimension(tf_idf, random_means,
                                        random_covs, 
                                        random_weights, cov_smoothing=1e-5)
print (out_random_init['loglik']) # print history of log-likelihood over time

## Quiz Question: Is the final loglikelihood larger or smaller than the final loglikelihood we obtained above when initializing EM with the results from running k-means?



## Quiz Question: For the above model, out_random_init, use the visualize_EM_clustersmethod you created above. Are the clusters more or less interpretable than the ones found after initializing using k-means?



In [ ]:
visualize_EM_clusters(tf_idf, out_random_init['means'],
                      out_random_init['covs'], map_index_to_word_2)